## Описание датасета

В работе используется объединённый датасет по публичным российским компаниям, который сочетает рыночные данные (цены акций, капитализация, биржевые мультипликаторы), фундаментальные показатели из T-Invest API и бухгалтерскую финансовую отчётность за 2024 год из базы СПАРК.

### Получение списка торгуемых акций (MOEX) через T-Invest API

В этом шаге мы выгружаем базовый список акций, доступных для торговли, отфильтровывая рублевые обыкновенные акции (без привилегированных), и сохраняем ключевые идентификаторы компаний в CSV.

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

SANDBOX_TOKEN = os.getenv("SANDBOX_TOKEN")

In [ ]:
import logging
import csv
from t_tech.invest import Client, InstrumentStatus

TOKEN = SANDBOX_TOKEN
logging.basicConfig(level=logging.INFO)

OUT_FILE = "shares_rub_common.csv"

rows = []

with Client(TOKEN) as client:
    resp = client.instruments.shares(
        instrument_status=InstrumentStatus.INSTRUMENT_STATUS_BASE,
        instrument_exchange=InstrumentStatus.INSTRUMENT_STATUS_UNSPECIFIED,
    )

    for s in resp.instruments:
        if s.currency == "rub" and "привил" not in s.name.lower():
            rows.append({
                "figi": s.figi,
                "ticker": s.ticker,
                "name": s.name,
                "issue_size": s.issue_size,
                "uid": s.uid,
                "assetUid": s.asset_uid,
            })

with open(OUT_FILE, "w", newline="", encoding="utf-8-sig") as f:
    writer = csv.DictWriter(
        f,
        fieldnames=["figi", "ticker", "name", "issue_size", "currency", "uid", "assetUid"],
    )
    writer.writeheader()
    writer.writerows(rows)

print(f"Saved {len(rows)} rows to {OUT_FILE}")


INFO:t_tech.invest.logging:dfe09614bca18e57b2f8f7a35b8061d9 Shares


Saved 151 rows to shares_rub_common.csv


### Загрузка дневных котировок по всем акциям и сохранение в отдельные файлы

На этом этапе для каждой акции из списка загружаются дневные свечи за последний год через T-Invest API, после чего данные сохраняются в отдельные CSV-файлы (по одной компании).


In [35]:
import csv
import logging
import os
import re
from datetime import timedelta
from pathlib import Path

from t_tech.invest import CandleInterval, Client
from t_tech.invest.caching.market_data_cache.cache import MarketDataCache
from t_tech.invest.caching.market_data_cache.cache_settings import MarketDataCacheSettings
from t_tech.invest.utils import now

TOKEN = SANDBOX_TOKEN  
logging.basicConfig(format="%(levelname)s: %(message)s", level=logging.FATAL)

INPUT_CSV = "shares_rub_common.csv"  
OUT_DIR = Path("stocks")
CACHE_DIR = Path("market_data_cache")

DAYS = 365 * 3
INTERVAL = CandleInterval.CANDLE_INTERVAL_DAY


def safe_filename(s):
    s = s.strip()
    s = re.sub(r'[<>:"/\\|?*\x00-\x1F]', "_", s) 
    s = re.sub(r"\s+", " ", s)
    return s[:150] if len(s) > 150 else s


def q_to_float(q):
    if q is None:
        return None
    try:
        return float(q.units) + float(q.nano) / 1_000_000_000
    except Exception:
        return None


def read_figis_from_csv(path):
    items = []
    with open(path, "r", encoding="utf-8-sig", newline="") as f:
        reader = csv.DictReader(f)
        for row in reader:
            figi = (row.get("figi") or "").strip()
            ticker = (row.get("ticker") or "").strip()
            name = (row.get("name") or "").strip()
            if figi:
                items.append((figi, ticker, name))
    return items


def save_candles_for_figi(market_data_cache, figi, out_path):
    candles = list(
        market_data_cache.get_all_candles(
            figi=figi,
            from_=now() - timedelta(days=DAYS),
            interval=INTERVAL,
        )
    )

    out_path.parent.mkdir(parents=True, exist_ok=True)
    with open(out_path, "w", encoding="utf-8-sig", newline="") as f:
        w = csv.writer(f)
        w.writerow(["time", "open", "high", "low", "close", "volume", "is_complete"])
        for c in candles:
            w.writerow([
                c.time.isoformat() if c.time else None,
                q_to_float(getattr(c, "open", None)),
                q_to_float(getattr(c, "high", None)),
                q_to_float(getattr(c, "low", None)),
                q_to_float(getattr(c, "close", None)),
                getattr(c, "volume", None),
                getattr(c, "is_complete", None),
            ])

    return len(candles)


In [ ]:
OUT_DIR.mkdir(parents=True, exist_ok=True)

items = read_figis_from_csv(INPUT_CSV)

with Client(TOKEN) as client:
    settings = MarketDataCacheSettings(base_cache_dir=CACHE_DIR)
    market_data_cache = MarketDataCache(settings=settings, services=client)

    ok = 0
    for figi, ticker, name in items:
        label = ticker or name or figi
        fname = safe_filename(f"{ticker or 'TICKER'}_{figi}.csv")
        out_path = OUT_DIR / fname

        try:
            n = save_candles_for_figi(market_data_cache, figi, out_path)
            logging.info("Saved %s candles for %s (%s) -> %s", n, label, figi, out_path)
            ok += 1
        except Exception as e:
            logging.exception("Failed for %s (%s): %s", label, figi, e)

print("Done")

Done


### Выгрузка фундаментальных показателей для всех компаний

На этом шаге мы батчами запрашиваем фундаментальные показатели (мультипликаторы и финансовые метрики) для всех компаний из списка, используя assetUid, и сохраняем результаты в единый CSV-файл.


In [23]:
import csv
import logging
from datetime import date, datetime
from pathlib import Path

from t_tech.invest import Client, GetAssetFundamentalsRequest

TOKEN = SANDBOX_TOKEN
logging.basicConfig(format="%(levelname)s: %(message)s", level=logging.INFO)

INPUT_CSV = "shares_rub_common.csv"
OUT_CSV = "asset_fundamentals_all.csv"

BATCH_SIZE = 50 


def iso(v):
    if isinstance(v, (datetime, date)):
        return v.isoformat()
    return v


def read_asset_uids(path):
    uids = []
    with open(path, "r", encoding="utf-8-sig", newline="") as f:
        r = csv.DictReader(f)

        possible_cols = ["assetUid", "asset_uid", "uid"]
        cols = r.fieldnames or []
        col = next((c for c in possible_cols if c in cols), None)
        if not col:
            pass

        for row in r:
            uid = (row.get(col) or "").strip()
            if uid:
                uids.append(uid)

    seen = set()
    out = []
    for u in uids:
        if u not in seen:
            seen.add(u)
            out.append(u)
    return out


def obj_to_row(obj):
    d = {}
    for k, v in vars(obj).items():
        d[k] = iso(v)
    return d


def chunked(xs, n):
    for i in range(0, len(xs), n):
        yield xs[i:i + n]


asset_uids = read_asset_uids(INPUT_CSV)
logging.info("Loaded asset UIDs: %d", len(asset_uids))

all_rows = []
with Client(TOKEN) as client:
    for batch in chunked(asset_uids, BATCH_SIZE):
        resp = client.instruments.get_asset_fundamentals(GetAssetFundamentalsRequest(assets=batch))

        for f in resp.fundamentals:
            all_rows.append(obj_to_row(f))

        logging.info("Fetched fundamentals: +%d (total %d)", len(resp.fundamentals), len(all_rows))

fieldnames = sorted({k for row in all_rows for k in row.keys()})

with open(OUT_CSV, "w", encoding="utf-8-sig", newline="") as f:
    w = csv.DictWriter(f, fieldnames=fieldnames)
    w.writeheader()
    w.writerows(all_rows)

logging.info("Saved %d rows to %s", len(all_rows), OUT_CSV)

INFO:root:Loaded asset UIDs: 151
INFO:t_tech.invest.logging:0d3034df8e37c1cdc857a4767d42b79f GetAssetFundamentals
INFO:root:Fetched fundamentals: +50 (total 50)
INFO:t_tech.invest.logging:9e65a0e049156ad4dcf0ddb92288d0fc GetAssetFundamentals
INFO:root:Fetched fundamentals: +49 (total 99)
INFO:t_tech.invest.logging:efe53df0b508aaea049ec8a4229ff65d GetAssetFundamentals
INFO:root:Fetched fundamentals: +48 (total 147)
INFO:t_tech.invest.logging:3490b8114fd5fea4b5c90faa63d7caca GetAssetFundamentals
INFO:root:Fetched fundamentals: +1 (total 148)
INFO:root:Saved 148 rows to asset_fundamentals_all.csv


### Формирование сводной таблицы с рыночными и фундаментальными показателями

На этом этапе мы объединяем список акций и фундаментальные данные в одну сводную таблицу по asset_uid, упорядочивая ключевые показатели и сохраняя результат в CSV.


In [27]:
import pandas as pd

SHARES_FILE = "shares_rub_common.csv"
FUND_FILE = "asset_fundamentals_all.csv"
OUT_FILE = "stocks_summary.csv"


shares = pd.read_csv(SHARES_FILE, encoding="utf-8-sig")
funds = pd.read_csv(FUND_FILE, encoding="utf-8-sig")

shares = shares.rename(columns={"assetUid": "asset_uid"})

df = shares.merge(
    funds,
    how="left",
    on="asset_uid",
)

preferred_cols = [
    "figi",
    "ticker",
    "name",
    "asset_uid",
    "issue_size",
    "shares_outstanding",
    "free_float",
    "market_capitalization",
    "pe_ratio_ttm",
    "price_to_book_ttm",
    "price_to_sales_ttm",
    "roe",
    "roa",
    "net_margin_mrq",
    "revenue_ttm",
    "net_income_ttm",
    "total_debt_mrq",
    "total_debt_to_equity_mrq",
    "average_daily_volume_last_10_days",
    "average_daily_volume_last_4_weeks",
    "high_price_last_52_weeks",
    "low_price_last_52_weeks",
    "currency",
]

first_cols = [c for c in preferred_cols if c in df.columns]

rest_cols = [c for c in df.columns if c not in first_cols]

df = df[first_cols + rest_cols]

df.to_csv(OUT_FILE, index=False, encoding="utf-8-sig")

print("Saved summary table:", OUT_FILE)
print("Rows:", len(df))
print("Columns:", len(df.columns))

missing = [c for c in preferred_cols if c not in df.columns]
if missing:
    print("Missing columns (ok):", missing)

Saved summary table: stocks_summary.csv
Rows: 151
Columns: 62
Missing columns (ok): ['currency']


### Объединение рыночных данных с бухгалтерской отчетностью СПАРК

На этом шаге сводная таблица с рыночными и фундаментальными показателями объединяется с финансовой отчетностью из СПАРКа по биржевому тикеру, при этом сохраняются только компании, присутствующие в обоих источниках, и устраняются дубликаты.

In [28]:
import pandas as pd

STOCKS_FILE = "stocks_summary.csv"
SPARK_FILE = "СПАРК_Выборка_компаний_20260204_1918.xlsx"
OUT_FILE = "stocks_summary_with_spark.csv"


stocks = pd.read_csv(STOCKS_FILE, encoding="utf-8-sig")
spark = pd.read_excel(SPARK_FILE)

spark = spark.rename(columns={"Тикер биржевой": "ticker"})

stocks["ticker"] = stocks["ticker"].astype(str).str.strip().str.upper()
spark["ticker"] = spark["ticker"].astype(str).str.strip().str.upper()

dup = spark[spark["ticker"].duplicated(keep=False)].copy()
if len(dup) > 0:
    print("SPARK duplicate tickers (count):")
    print(dup["ticker"].value_counts().head(30).to_string())
    print()

sort_cols = []
if "2024, Активы  всего, RUB" in spark.columns:
    sort_cols.append("2024, Активы  всего, RUB")
elif "2024, Выручка, RUB" in spark.columns:
    sort_cols.append("2024, Выручка, RUB")

if sort_cols:
    spark_sorted = spark.sort_values(sort_cols, ascending=False)
    spark_uniq = spark_sorted.drop_duplicates(subset=["ticker"], keep="first")
else:
    spark_uniq = spark.drop_duplicates(subset=["ticker"], keep="first")

stocks_uniq = stocks.drop_duplicates(subset=["ticker"], keep="first")

df = stocks_uniq.merge(
    spark_uniq,
    how="inner",
    on="ticker",
    suffixes=("_api", "_spark"),
    validate="one_to_one" 
)

df.to_csv(OUT_FILE, index=False, encoding="utf-8-sig")

print("Saved merged table:", OUT_FILE)
print("Rows:", len(df))
print("Columns:", len(df.columns))


SPARK duplicate tickers (count):
ticker
NAN    3779

Saved merged table: stocks_summary_with_spark.csv
Rows: 91
Columns: 154


### Финальный датасет

На этом этапе мы выбираем нужные нам переменные для финального датасета и приводим значения к читаемому формату

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

IN_FILE = Path("stocks_summary_with_spark.csv")  
OUT_XLSX = Path("final_dataset.xlsx")

df = pd.read_csv(IN_FILE, encoding="utf-8-sig")

df = df.replace(r"^\s*$", np.nan, regex=True)
df = df.dropna(axis=1, how="all")

important_cols = [
    "ticker", "name",

    "last_close", "last_close_time",
    "market_capitalization",
    "pe_ratio_ttm", "price_to_book_ttm", "price_to_sales_ttm",
    "roe", "roa", "net_margin_mrq",
    "beta", "free_float",
    "average_daily_volume_last_4_weeks",

    "Код налогоплательщика",
    "2024, Среднесписочная численность работников",

    "2024, Выручка, RUB",
    "2024, Чистая прибыль (убыток), RUB",
    "2024, Активы  всего, RUB",
    "2024, Капитал и резервы, RUB",
    "2024, Совокупный долг, RUB",
]

keep = [c for c in important_cols if c in df.columns]
df = df[keep].copy()

num_candidates = [
    "last_close", "market_capitalization",
    "pe_ratio_ttm", "price_to_book_ttm", "price_to_sales_ttm",
    "roe", "roa", "net_margin_mrq",
    "beta", "free_float", "average_daily_volume_last_4_weeks",
    "2024, Среднесписочная численность работников",
    "2024, Выручка, RUB",
    "2024, Чистая прибыль (убыток), RUB",
    "2024, Активы  всего, RUB",
    "2024, Капитал и резервы, RUB",
    "2024, Совокупный долг, RUB",
]
for c in num_candidates:
    if c in df.columns:
        df[c] = (
            df[c].astype(str)
            .str.replace("\u00A0", "", regex=False)  
            .str.replace(" ", "", regex=False)
            .str.replace(",", ".", regex=False)
        )
        df[c] = pd.to_numeric(df[c], errors="coerce")

date_cols = [c for c in ["last_close_time"] if c in df.columns]
for c in date_cols:
    df[c] = pd.to_datetime(df[c], errors="coerce", utc=True).dt.strftime("%Y-%m")

rename_map = {
    "ticker": "Тикер",
    "name": "Компания",
    "last_close": "Цена закрытия (последняя), RUB",
    "last_close_time": "Период (YYYY-MM)",
    "market_capitalization": "Капитализация, RUB",
    "pe_ratio_ttm": "P/E (TTM)",
    "price_to_book_ttm": "P/B (TTM)",
    "price_to_sales_ttm": "P/S (TTM)",
    "roe": "ROE",
    "roa": "ROA",
    "net_margin_mrq": "Чистая маржа (MRQ)",
    "beta": "Бета",
    "free_float": "Free-float",
    "average_daily_volume_last_4_weeks": "Средн. дневной объём (4 недели)",
    "Код налогоплательщика": "ИНН",
    "2024, Среднесписочная численность работников": "Ср. численность 2024",
    "2024, Выручка, RUB": "Выручка 2024, RUB",
    "2024, Чистая прибыль (убыток), RUB": "Чистая прибыль 2024, RUB",
    "2024, Активы  всего, RUB": "Активы 2024, RUB",
    "2024, Капитал и резервы, RUB": "Капитал и резервы 2024, RUB",
    "2024, Совокупный долг, RUB": "Совокупный долг 2024, RUB",
}
df = df.rename(columns={k: v for k, v in rename_map.items() if k in df.columns})

df = df.dropna(axis=1, how="all")

with pd.ExcelWriter(OUT_XLSX, engine="openpyxl") as writer:
    df.to_excel(writer, index=False, sheet_name="dataset")

print(f"Saved: {OUT_XLSX} | rows={len(df)} cols={len(df.columns)}")
df.head()


Saved: final_dataset.xlsx | rows=91 cols=19


,Тикер,Компания,"Капитализация, RUB",P/E (TTM),P/B (TTM),P/S (TTM),ROE,ROA,Чистая маржа (MRQ),Бета,Free-float,Средн. дневной объём (4 недели),ИНН,Ср. численность 2024,"Выручка 2024, RUB","Чистая прибыль 2024, RUB","Активы 2024, RUB","Капитал и резервы 2024, RUB","Совокупный долг 2024, RUB"
0,VSMO,ВСМПО-АВИСМА,3.611051e+11,52.42,1.20,3.34,2.26,1.46,6.36,1.00,0.10,1.285360e+03,6.607001e+09,13486.0,1.010067e+11,1.003488e+10,4.662586e+11,2.736734e+11,1.925853e+11
1,UNAC,Объединенная авиастроительная корпорация,4.860102e+11,0.00,4.89,0.81,-15.40,-0.54,-2.06,0.56,0.03,1.778768e+08,7.708619e+09,33340.0,1.980137e+11,-2.412211e+10,1.862248e+12,3.292258e+11,1.533022e+12
2,MGNT,Магнит,3.236705e+11,11.97,1.76,0.10,15.68,1.75,0.83,0.61,0.67,2.640095e+05,2.309086e+09,4500.0,4.120620e+08,6.377790e+10,3.077855e+11,2.081270e+11,9.965849e+10
3,KZIZ,Красногорский завод им. С.А. Зверева - ао,1.568798e+09,0.00,0.18,0.07,-17.79,-3.15,-7.03,0.56,0.00,3.875800e+02,5.024023e+09,NaN,1.881731e+10,3.247110e+08,4.907697e+10,7.375392e+09,4.170158e+10
4,SELG,Селигдар,5.983270e+10,0.00,4.30,0.77,-64.10,-5.67,-13.30,0.58,0.25,3.236956e+07,1.402047e+09,241.0,1.173288e+09,6.420100e+07,1.175407e+11,3.076840e+10,8.677229e+10


### Парсинг новостей с Финам для дальнейшего текстового анализа

In [ ]:
import csv
import re
import time
from pathlib import Path
from urllib.parse import urljoin

import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


STOCKS_FILE = "stocks_summary_with_spark.csv"
OUT_DIR = Path("parsed_news")
OUT_DIR.mkdir(parents=True, exist_ok=True)

BASE = "https://www.finam.ru"
URL_TEMPLATE = "https://www.finam.ru/quote/moex/{ticker}/publications/"

WAIT_SEC = 20
SLEEP_BETWEEN_TICKERS = 1.0

CLICK_LOAD_MORE_TIMES = 0  


def safe_filename(name: str) -> str:
    name = (name or "").strip()
    name = re.sub(r'[<>:"/\\|?*\x00-\x1F]', "_", name)
    name = re.sub(r"\s+", " ", name)
    return name[:150] if len(name) > 150 else name


RU_MONTHS = {
    "янв": "01", "фев": "02", "мар": "03", "апр": "04", "май": "05", "июн": "06",
    "июл": "07", "авг": "08", "сен": "09", "окт": "10", "ноя": "11", "дек": "12",
}


def parse_date(raw: str):
    raw = (raw or "").strip().lower()

    m = re.search(r"(\d{2})\.(\d{2})\.(\d{2,4})", raw)
    if m:
        dd, mm, yy = m.group(1), m.group(2), m.group(3)
        if len(yy) == 2:
            yy = "20" + yy
        return f"{yy}-{mm}-{dd}", f"{yy}-{mm}"

    m = re.search(r"(\d{1,2})\s+([а-яё]{3,})\s+(\d{4})", raw)
    if m:
        dd = f"{int(m.group(1)):02d}"
        mon = m.group(2)[:3]
        yy = m.group(3)
        mm = RU_MONTHS.get(mon)
        if mm:
            return f"{yy}-{mm}-{dd}", f"{yy}-{mm}"

    return None, None


def parse_publications_from_html(html: str, page_url: str, ticker: str):
    soup = BeautifulSoup(html, "lxml")
    items = []

    for node in soup.select('div[data-publication]'):
        date_raw = ""
        date_span = node.select_one("div > div > span.font-xs.cl-darkgrey")
        if date_span:
            date_raw = date_span.get_text(strip=True)
        date_full, date_ym = parse_date(date_raw)

        a = node.select_one("a.cl-blue.font-l.bold")
        title = a.get_text(strip=True) if a else ""
        href = a.get("href") if a else ""
        url = urljoin(BASE, href) if href else ""

        p = node.select_one("p.font-s.cl-black")
        text = p.get_text(" ", strip=True) if p else ""

        if not (title or text or url):
            continue

        items.append({
            "ticker": ticker,
            "date": date_full,
            "date_ym": date_ym,
            "date_raw": date_raw,
            "title": title,
            "text": text,
            "url": url,
            "source_page": page_url,
        })

    return items


def setup_driver():
    options = Options()

    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--window-size=1400,900")
    options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/122.0.0.0 Safari/537.36"
    )
    return webdriver.Chrome(options=options)


def click_load_more_if_exists(driver, times: int):
    for _ in range(times):
        try:
            btn = driver.find_element(By.XPATH, "//button[contains(., 'Показать') or contains(., 'Загрузить')]")
            driver.execute_script("arguments[0].scrollIntoView(true);", btn)
            time.sleep(0.4)
            btn.click()
            time.sleep(1.2)
        except Exception:
            break


def main():
    df = pd.read_csv(STOCKS_FILE, encoding="utf-8-sig")
    tickers = (
        df["ticker"]
        .astype(str)
        .str.strip()
        .str.upper()
        .dropna()
        .unique()
        .tolist()
    )

    driver = setup_driver()
    wait = WebDriverWait(driver, WAIT_SEC)

    try:
        for t in tickers:
            url = URL_TEMPLATE.format(ticker=t.lower())
            driver.get(url)

            try:
                wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div[data-publication]")))
            except Exception:
                out_file = OUT_DIR / f"{safe_filename(t)}_finam_publications.csv"
                with open(out_file, "w", newline="", encoding="utf-8-sig") as f:
                    w = csv.DictWriter(
                        f,
                        fieldnames=["ticker", "date", "date_ym", "date_raw", "title", "text", "url", "source_page"]
                    )
                    w.writeheader()
                print(f"[{t}] no publications found -> {out_file}")
                time.sleep(SLEEP_BETWEEN_TICKERS)
                continue

            if CLICK_LOAD_MORE_TIMES > 0:
                click_load_more_if_exists(driver, CLICK_LOAD_MORE_TIMES)

            html = driver.page_source
            items = parse_publications_from_html(html, url, t)

            out_file = OUT_DIR / f"{safe_filename(t)}_finam_publications.csv"
            with open(out_file, "w", newline="", encoding="utf-8-sig") as f:
                w = csv.DictWriter(
                    f,
                    fieldnames=["ticker", "date", "date_ym", "date_raw", "title", "text", "url", "source_page"]
                )
                w.writeheader()
                w.writerows(items)

            print(f"[{t}] saved {len(items)} -> {out_file}")
            time.sleep(SLEEP_BETWEEN_TICKERS)

    finally:
        driver.quit()


if __name__ == "__main__":
    main()


[VSMO] saved 50 -> parsed_news\VSMO_finam_publications.csv
[UNAC] saved 50 -> parsed_news\UNAC_finam_publications.csv
[MGNT] saved 50 -> parsed_news\MGNT_finam_publications.csv
[KZIZ] saved 1 -> parsed_news\KZIZ_finam_publications.csv
[SELG] saved 50 -> parsed_news\SELG_finam_publications.csv
[SLAV] saved 4 -> parsed_news\SLAV_finam_publications.csv
[PRFN] saved 48 -> parsed_news\PRFN_finam_publications.csv
[NKHP] saved 50 -> parsed_news\NKHP_finam_publications.csv
[TGKJ] saved 50 -> parsed_news\TGKJ_finam_publications.csv
[ALRS] saved 50 -> parsed_news\ALRS_finam_publications.csv
[TATN] saved 50 -> parsed_news\TATN_finam_publications.csv
[GRNT] saved 50 -> parsed_news\GRNT_finam_publications.csv
[RTKM] saved 50 -> parsed_news\RTKM_finam_publications.csv
[TGKN] saved 50 -> parsed_news\TGKN_finam_publications.csv
[GCHE] saved 50 -> parsed_news\GCHE_finam_publications.csv
[UWGN] saved 50 -> parsed_news\UWGN_finam_publications.csv
[NKNC] saved 50 -> parsed_news\NKNC_finam_publications.csv